In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
#import wget
import sys
sys.path.append('../')
import argparse
import json
import pandas as pd
import random
import numpy as np
import string
import nltk
from functools import partial
import re
from  tqdm import tqdm
import torch
nltk.download('punkt')
from functools import partial
import nltk
from src.dataset_processor import load_all_data
from src.utils import SmartCollator, get_args, setuptokenizer
from src.dataset_processor import (
    Multi_taskQuestionGenerationDataset as QuestionGenerationDataset,
)
from src.model_utils import CustomTrainer, get_training_arguments, model_init
from src.config import DATASET_PATH, GenerationTasks
from transformers.trainer_callback import EarlyStoppingCallback

[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
model_base = 'facebook/bart-base'
tokenizer = setuptokenizer(model_base=model_base,
                           special_tokens=[
            GenerationTasks.vanilla_question_gen,
            GenerationTasks.context_question_gen,
            GenerationTasks.question_paraphrase,
            "<section>",
            "</section>",
        ],
                           )

In [6]:
from src.model_utils import model_init
saved_model_path = '../trained_models_mtl/bart_base_model_1/'
#'../trained_models_mtl/bart_base_model_1/checkpoint-25524//'
trained_weights = torch.load(f'{saved_model_path}checkpoint-127620/pytorch_model.bin')

generator = model_init(model_base=model_base,
                       vocab_size=len(tokenizer))
generator.load_state_dict(trained_weights)
device = generator.device

In [7]:
# Load argparse elements
import pickle as pk
args = pk.load(open(f'{saved_model_path}/train_args.ap','rb'))

In [8]:
args

Namespace(model_base='facebook/bart-base', output_dir='trained_models_mtl/', run_id='bart_base_model_1', eval_steps=1000, learning_rate=3e-05, max_squad_size=80000, max_seq_len=512, evaluation_strategy='epoch', save_strategy='epoch', seed=10, lr_scheduler_type='cosine', weight_decay=0.3, warmup_ratio=0.25, num_train_epochs=5, save_total_limit=1, per_device_train_batch_size=16, per_device_eval_batch_size=16, verbose=False)

In [9]:
from transformers import TrainingArguments, Trainer
train_args = TrainingArguments(push_to_hub=True,
                               hub_model_id="kaejo98/bart-base_question_generation",
                               overwrite_output_dir=True,
                               output_dir='trained_models/setup_1/',
                               evaluation_strategy=args.evaluation_strategy,  # "epoch",
                               save_strategy=args.save_strategy,  # 'epoch',
                               lr_scheduler_type=args.lr_scheduler_type,
                               learning_rate=args.learning_rate,

                               save_total_limit=args.save_total_limit,
                               weight_decay=args.weight_decay,
                               warmup_ratio=args.warmup_ratio,
                               num_train_epochs=args.num_train_epochs,
                               per_device_train_batch_size=args.per_device_train_batch_size,
                               per_device_eval_batch_size=args.per_device_train_batch_size,
                               disable_tqdm=not args.verbose,
                               eval_steps=args.eval_steps,
                               save_steps=args.eval_steps,
                               )


In [10]:
trainer = Trainer(generator,train_args,tokenizer=tokenizer)

Cloning https://huggingface.co/kaejo98/bart-base_question_generation into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/532M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.37k/3.37k [00:00<?, ?B/s]

Clean file training_args.bin:  30%|##9       | 1.00k/3.37k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/532M [00:00<?, ?B/s]

In [11]:
trainer.model.load_state_dict(trained_weights)

<All keys matched successfully>

In [12]:
trainer.push_to_hub("New questions generation")

Saving model checkpoint to trained_models/setup_1/
Configuration saved in trained_models/setup_1/config.json
Model weights saved in trained_models/setup_1/pytorch_model.bin
tokenizer config file saved in trained_models/setup_1/tokenizer_config.json
Special tokens file saved in trained_models/setup_1/special_tokens_map.json
added tokens file saved in trained_models/setup_1/added_tokens.json


Upload file pytorch_model.bin:   0%|          | 32.0k/532M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.37k/3.37k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/kaejo98/bart-base_question_generation
   120358f..dc51d72  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/kaejo98/bart-base_question_generation
   dc51d72..ba60400  main -> main



'https://huggingface.co/kaejo98/bart-base_question_generation/commit/dc51d723a747e58c04767205045c1b85cb2e9bc9'

In [13]:
DATASET_PATH = '../curated_data/'
train_data_packet = load_all_data(DATASET_PATH, mode="train")
test_data_packet = load_all_data(DATASET_PATH, mode="dev")+load_all_data(DATASET_PATH, mode="test")

processing files:  ['../curated_data/drop_train.csv', '../curated_data/squad_train.csv', '../curated_data/rope_train.csv', '../curated_data/extra_data_train.csv', '../curated_data/sci_train.csv']
processing files:  ['../curated_data/squad_dev.csv', '../curated_data/drop_dev.csv', '../curated_data/rope_dev.csv', '../curated_data/sci_dev.csv']
processing files:  ['../curated_data/sci_test.csv']


In [17]:
len(test_data_packet)//2

21734

In [1]:
from huggingface_hub import notebook_login

In [3]:
notebook_login()

Login successful
Your token has been saved to /home/nlplab/.huggingface/token


In [1]:
3e-4

0.0003